# Decoder-only based GPT (language model)

Here we take a transformer block, the decoder in particular, and use it for the task of language modeling. In general, this is how GPTs are trained. We will do this on a much smaller scale.

In [33]:
import torch
from torch.utils.data import random_split
import sys 
sys.path.append("../models")
from torchtext.datasets import EnWik9
from torchtext.data.functional import filter_wikipedia_xml

In [87]:
for i in range(7):
    book_num = i+1
    with open('../data/harry_potter_1.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    print(len(text))

439742


In [34]:
dataset_iter = EnWik9()

In [35]:
dataset = list(dataset_iter)

In [36]:
dataset[-4]

'==Complications=='

In [37]:
data_iter = EnWik9()
filter_data_iter = filter_wikipedia_xml(data_iter)  # convert from xml to pure text 


In [38]:
filtered_dataset = list(filter_data_iter)

In [39]:
for i in range(5):
    print(filtered_dataset[i])

anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists
the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic institutio

In [67]:
filtered_dataset = filtered_dataset[:int(len(filtered_dataset) * 0.1)]

In [68]:
len(filtered_dataset)

633671

In [69]:
train_val_size = int(len(filtered_dataset) * 0.8)  
test_size = len(filtered_dataset) - train_val_size
train_val_data, test_data = random_split(filtered_dataset, [train_val_size, test_size])

In [70]:
train_size = int(len(train_val_data) * 0.8)  
val_size = len(train_val_data) - train_size
train_data, test_data = random_split(train_val_data, [train_size, val_size])

In [71]:
print(f"train set size: {train_size}, test: {test_size}, val: {val_size}")

train set size: 405548, test: 126735, val: 101388
